# Define the bi-linearly product
* object: vecSym, scalarTimes
* mathod: dot product :  $dot(a \mathbf{p},\mathbf{\epsilon})=a \mathbf{p}\cdot\mathbf{\epsilon}$
* coefficient of a monomial:$Coefficient(f,m)$

In [1]:
using SymEngine

In [2]:
versioninfo()

Julia Version 1.1.0
Commit 80516ca202 (2019-01-21 21:24 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


In [3]:
struct vecSym <: Number
    name::String
    function vecSym(x::String)
        new(x)
    end
end

In [4]:
function Base.show(io::IO, vec::vecSym)
    print(io, ("\033[1m"*vec.name*"\x1b[0m"))
               # Black \033[1m;Oringe:\x1b[1;31m;normal:\x1b[0m
end

In [5]:
struct scalarTimes
    scalar::Basic
    vec::vecSym
    function scalarTimes(x::Basic,y::vecSym)
        new(x,y)
    end
end
function Base.show(io::IO, y::scalarTimes)
    print(io,y.scalar,y.vec)
end
struct vecAdd
    veca
    function vecAdd(x)
        new(x)
    end
end

function Base.show(io::IO, y::vecAdd)
    print(io, "+","(",y.veca,")")
end

In [6]:
import Base.*
*(x::Basic,y::vecSym)= scalarTimes(x,y)
import Base.+
+(x::vecSym,y::vecSym)= vecAdd((x,y))
+(x::vecSym,y...)= vecAdd((x,y...))
+(x::scalarTimes,y::vecSym)= vecAdd((x,y))
+(x::scalarTimes,y::scalarTimes)= vecAdd((x,y))
+(x::scalarTimes,y...)= vecAdd((x,y...))
+(x::Tuple)= vecAdd(x)

+ (generic function with 172 methods)

In [7]:
function dot(x::vecSym,y::vecSym)
    xn=x.name;
    yn=y.name;
    if x.name<y.name 
        return symbols("$x"*"⋅"*"$y")
    else
        return symbols("$y"*"⋅"*"$x")
    end
end

dot (generic function with 1 method)

In [102]:
function dot(x::scalarTimes,y::vecSym)
    x.scalar*dot(x.vec,y)
end
function dot(y::vecSym,x::scalarTimes)
    dot(x::scalarTimes,y::vecSym)
end
function dot(x::vecAdd,y::vecSym)
    xx = x.veca;
    s=0;
    for i=1:length(xx)
       s+=dot(xx[i],y)
    end
    return s
end
function dot(y::vecSym,x::vecAdd)
    dot(x::vecAdd,y::vecSym)
end
function dot(x::vecAdd,y::vecAdd)
    yy = y.veca;
    s=0;
    for i=1:length(yy)
       s+=dot(x,yy[i])
    end
    return s
end
function dot(x::vecAdd,y::scalarTimes)
   y.scalar*dot(x,y.vec)
end
function dot(x::Tuple, y::vecSym)
    # s = 0
   # for i in x
   #    s+=dot(x[i],y)
   # end
    s = 0
    for i=1:length(x)
       s+=dot(x[i],y)
    end
    return s
end

dot (generic function with 8 methods)

In [9]:
p_1=vecSym("P₁");p_2=vecSym("P₂");p_3=vecSym("P₃");p_4=vecSym("P₄");p_5=vecSym("P₅");p_6=vecSym("P₆");
p_7=vecSym("P₇");
ϵ_1=vecSym("ϵ₁");
ϵ_2=vecSym("ϵ₂");ϵ_3=vecSym("ϵ₃");ϵ_4=vecSym("ϵ₄");ϵ_5=vecSym("ϵ₅");ϵ_6=vecSym("ϵ₆");ϵ_7=vecSym("ϵ₇");
a_1=symbols("a₁");
a_2=symbols("a₂");
a_3=symbols("a₃");

#### Examples

In [10]:
tt=dot(ϵ_1,p_2)dot(ϵ_3,p_2)dot(ϵ_4,ϵ_7)dot(ϵ_5,p_1)dot(ϵ_6,p_1);

In [11]:
p_1.name[2]==ϵ_1.name[3]

true

In [92]:
(a_1*p_1+a_2*p_2+a_1*p_3+a_2*p_4).veca[1]

a₁P₁

In [104]:
expand(dot(a_1*p_1+a_2*p_2+a_1*p_3+a_2*p_4,a_1*ϵ_1+a_2*ϵ_2))

a₁^2*P₁⋅ϵ₁ + a₁^2*P₃⋅ϵ₁ + a₂^2*P₂⋅ϵ₂ + a₂^2*P₄⋅ϵ₂ + a₁*a₂*P₁⋅ϵ₂ + a₁*a₂*P₂⋅ϵ₁ + a₁*a₂*P₃⋅ϵ₂ + a₁*a₂*P₄⋅ϵ₁

#### Try to get the coefficients for the dot product monomials

In [70]:
tt1list=get_args(tt1);

In [51]:
dump(tt1)

Basic
  ptr: Ptr{Nothing} @0x00007fd3ce8664b0


In [48]:
s_1=Symbol("s1")
s_2=Symbol("s2")

:s2

In [50]:
dump(:(2+a*b-c))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol -
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 2
        3: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol *
            2: Symbol a
            3: Symbol b
    3: Symbol c


In [90]:
free_symbols(tt1)

2-element Array{Basic,1}:
 a₁
 a₂

In [88]:
tt2=convert(Expr,tt1list[2])

:(10 * a₁ * a₂ ^ 9)

In [81]:
tt3=dump(tt2)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol ^
    2: Symbol a₂
    3: Int64 10


In [89]:
tt2.args

4-element Array{Any,1}:
   :*       
 10         
   :a₁      
   :(a₂ ^ 9)